In [1]:
import pandas as pd
import numpy as np
import spacy
from nltk.corpus import stopwords
import re
spacy_nlp = spacy.load('fr_core_news_sm')
import gensim, spacy
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
COLS_TO_REMOVE_GDN = ['authorId', 'authorType', 'authorZipCode', 'createdAt', 'id', 'publishedAt', 'reference', 'title', 'trashed', 'trashedStatus', 'updatedAt', 'Tomerge1','Tomerge2']

In [3]:
df = pd.read_csv("data/GDN/LA_TRANSITION_ECOLOGIQUE.csv", low_memory=False, nrows=100)
df = df.drop(COLS_TO_REMOVE_GDN, axis=1)

In [4]:
cols_name = list(df)[1:] # on retire la cols "id"
n_cols = len(cols_name)
cols_data = [None] * n_cols
for i in range(n_cols):
    col_i = df[cols_name[i]]
    cols_data[i] = col_i[~pd.isnull(col_i)]
data = [i.tolist() for i in cols_data]

In [5]:
emptycols = []
maxtext = 0
maxtext_ind = 0
for i in range(len(data)):
    if not len(data[i]):
        emptycols.append(i)
    else:
#         print(data[i])
#         print(maxtext, (" ".join(data[i])))
        if maxtext < len(" ".join(data[i])):
            maxtext_ind = i
            maxtext = len(" ".join(data[i]))
#             print(len(" ".join(data[i])), i)
print(maxtext, maxtext_ind)
# print(emptycols)

23970 37


In [17]:
data_c = data[maxtext_ind]

data_c = [re.sub('/\s\s+/g', ' ', sent) for sent in data_c]
data_c = [re.sub('\n', ' ', sent) for sent in data_c]
data_c = [re.sub('»', ' ', sent) for sent in data_c]
data_c = [re.sub('«', ' ', sent) for sent in data_c]
data_c = [re.sub(',', ' ', sent) for sent in data_c]
data_c = [re.sub('\’', ' ', sent) for sent in data_c]
data_c = [re.sub('\.+', ' ', sent) for sent in data_c]


<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \.
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \.
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:9: DeprecationWarning: invalid escape sequence \.
<ipython-input-17-87fc07eac147>:3: DeprecationWarning: invalid escape sequence \s
  data_c = [re.sub('/\s\s+/g', ' ', sent) for sent in data_c]
<ipython-input-17-87fc07eac147>:9: DeprecationWarning: invalid escape sequence \.
  data_c = [re.sub('\.+', ' ', sent) for sent in data_c]


In [24]:
clean_data = [[token.lemma_ for token in spacy_nlp(doc) if not(token.is_space or token.is_stop or len(token.text) < 3)] for doc in data_c]


[['problème',
  'trouve',
  'confronter',
  'ensemble',
  'planète',
  'dénoncent',
  'parfaire',
  'désordre',
  'gilet',
  'jaune',
  'France',
  'devoir',
  'surpopulation',
  'mondial',
  'population',
  'passer',
  'milliard',
  'habitant',
  '1900',
  'milliard',
  '2020',
  'monter',
  'bientôt',
  'milliard',
  '2040',
  'progrès',
  'communication',
  'village',
  'mondial',
  'individu',
  'fin',
  'fondre',
  'Asie',
  'fin',
  'fondre',
  'Afrique',
  'passer',
  'quartier',
  'campagne',
  'pays',
  'aspir',
  'vivre',
  'blâmer',
  'mal',
  'lotir',
  'concitoyen',
  'logement',
  'nourriture',
  'bien',
  'consommation',
  'déplacemer',
  'mère',
  'problème',
  'cas',
  'solution',
  'problème',
  'stabilisation',
  'croissance',
  'démographique',
  'partager',
  'richesse',
  'partager',
  'terre',
  'partage',
  'eau',
  'protection',
  'biodiversité',
  'règlemer',
  'conflit',
  'lutte',
  'déforestation',
  'lutte',
  'dérèglement',
  'climatique',
  'règlemer',
 

In [26]:
from collections import Counter
count_ents = Counter({})
print(count_ents)
for doc in clean_data:
    sp = spacy_nlp(doc)
    if len(sp.ents) > 0:
        count_ents += Counter([(x.text,x.label_) for x in sp.ents])

count_ents.most_common()[:10]

Counter()


[(('France', 'LOC'), 14),
 (('co2', 'MISC'), 8),
 (('commun', 'LOC'), 7),
 (('européen', 'LOC'), 4),
 (('fossile', 'MISC'), 4),
 (('baisser', 'PER'), 4),
 (('bouteille', 'LOC'), 4),
 (('html', 'MISC'), 4),
 (('état', 'LOC'), 4),
 (('gilet', 'PER'), 3)]